In [5]:
import pyodbc

conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=DESKTOP-EB31CGU\\SQLEXPRESS;'
    'DATABASE=bill;'
    'Trusted_Connection=yes;'
)

print("Connected to bill database successfully!")

cursor = conn.cursor()


Connected to bill database successfully!


In [6]:
cursor.execute("SELECT * FROM menu")

print("MENU TABLE:")
for row in cursor:
    print(row)


MENU TABLE:
(1, 'Burger', Decimal('120.00'))
(2, 'Pizza', Decimal('250.00'))
(3, 'Pasta', Decimal('180.00'))
(4, 'Sandwich', Decimal('80.00'))
(5, 'Coffee', Decimal('90.00'))


In [7]:
cursor.execute("SELECT * FROM orders")

print("\nORDERS TABLE:")
for row in cursor:
    print(row)



ORDERS TABLE:
(101, 1, 2, Decimal('240.00'))
(102, 2, 1, Decimal('250.00'))
(103, 3, 2, Decimal('360.00'))
(104, 5, 3, Decimal('270.00'))
(105, 4, 1, Decimal('80.00'))


In [14]:
import pyodbc

conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=DESKTOP-EB31CGU\\SQLEXPRESS;'
    'DATABASE=bill;'
    'Trusted_Connection=yes;'
)

cursor = conn.cursor()

print("\n------ MENU ------")
cursor.execute("SELECT * FROM menu")
for row in cursor:
    print(f"ID: {row.menu_id} | Item: {row.menu_name} | Price: {row.price}")

# 🔹 Take User Input
item_id = int(input("\nEnter Item ID: "))
quantity = int(input("Enter Quantity: "))

# 🔹 Fetch price from database
cursor.execute("SELECT menu_name, price FROM menu WHERE menu_id = ?", item_id)
item = cursor.fetchone()

if item:
    item_name = item.menu_name
    price = item.price
    total = price * quantity

    # 🔹 Generate Order ID automatically
    cursor.execute("SELECT ISNULL(MAX(orders_id),100) + 1 FROM orders")
    order_id = cursor.fetchone()[0]

    # 🔹 Insert into orders table
    cursor.execute(
        "INSERT INTO orders VALUES (?, ?, ?, ?)",
        order_id, item_id, quantity, total
    )
    conn.commit()

    # 🔹 Print Bill
    print("\n--------- BILL ---------")
    print(f"Order ID : {order_id}")
    print(f"Item     : {item_name}")
    print(f"Price    : {price}")
    print(f"Quantity : {quantity}")
    print("------------------------")
    print(f"Total    : {total}")
    print("------------------------")

else:
    print("Invalid Item ID!")

conn.close()



------ MENU ------
ID: 1 | Item: Burger | Price: 120.00
ID: 2 | Item: Pizza | Price: 250.00
ID: 3 | Item: Pasta | Price: 180.00
ID: 4 | Item: Sandwich | Price: 80.00
ID: 5 | Item: Coffee | Price: 90.00

--------- BILL ---------
Order ID : 107
Item     : Pizza
Price    : 250.00
Quantity : 3
------------------------
Total    : 750.00
------------------------


In [18]:
import pyodbc

conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=DESKTOP-EB31CGU\\SQLEXPRESS;'
    'DATABASE=bill;'
    'Trusted_Connection=yes;'
)

cursor = conn.cursor()

print("\n------ MENU ------")
cursor.execute("SELECT * FROM menu")
for row in cursor:
    print(f"ID: {row.menu_id} | Item: {row.menu_name} | Price: {row.price}")

grand_total = 0
bill_items = []

while True:
    item_id = int(input("\nEnter Item ID (0 to finish): "))

    if item_id == 0:
        break

    quantity = int(input("Enter Quantity: "))

    cursor.execute("SELECT menu_name, price FROM menu WHERE menu_id = ?", item_id)
    item = cursor.fetchone()

    if item:
        item_name = item.menu_name
        price = item.price
        total = price * quantity
        grand_total += total

        # Generate Order ID automatically
        cursor.execute("SELECT ISNULL(MAX(orders_id),100) + 1 FROM orders")
        order_id = cursor.fetchone()[0]

        # Insert into database
        cursor.execute(
            "INSERT INTO orders VALUES (?, ?, ?, ?)",
            order_id, item_id, quantity, total
        )
        conn.commit()

        bill_items.append((item_name, price, quantity, total))

        print("Item added successfully!")
    else:
        print("Invalid Item ID!")

# 🧾 Print Final Bill
print("\n\n========== FINAL BILL ==========")
print("Item        Price   Qty   Total")
print("---------------------------------")

for item in bill_items:
    print(f"{item[0]:10} {item[1]:6} {item[2]:5} {item[3]:7}")

print("---------------------------------")
print(f"Grand Total: {grand_total}")
print("=================================")

conn.close()



------ MENU ------
ID: 1 | Item: Burger | Price: 120.00
ID: 2 | Item: Pizza | Price: 250.00
ID: 3 | Item: Pasta | Price: 180.00
ID: 4 | Item: Sandwich | Price: 80.00
ID: 5 | Item: Coffee | Price: 90.00
Item added successfully!
Item added successfully!


========== FINAL BILL ==========
Item        Price   Qty   Total
---------------------------------
Burger     120.00     2  240.00
Pasta      180.00     4  720.00
---------------------------------
Grand Total: 960.00
